In [353]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [354]:
log_file_path = "../results/mascot_skyline_updated_100_nes_1.log"


In [355]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[2]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [356]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [357]:
Ne_skyline.head()

AttributeError: 'dict' object has no attribute 'head'

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
            if "\n" in i.split(".")[2]:
                interval = i.split(".")[2][0:2]
            else:
                interval = i.split(".")[2]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [ ]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

burnin_percent = 0.5
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()
Ne_df.head()

In [ ]:
exp_ne_df = Ne_df.copy()
# Select only numeric columns excluding the first two columns
numeric_columns = exp_ne_df.columns[1:]  # Exclude the first two columns

# Apply the exponential function only to numeric columns
exp_ne_df[numeric_columns] = exp_ne_df[numeric_columns].apply(pd.to_numeric, errors='coerce').apply(np.exp)

exp_ne_df

# exp_ne_df= np.exp(Ne_df.iloc[:, 1:])
# exp_ne_df

In [ ]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
            if "\n" in i.split(".")[2]:
                interval = i.split(".")[2][0:2]
            else:
                interval = i.split(".")[2]
            next_interval = int(interval)+2 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+"."+ str(deme) +".diff." + str(interval)] = (29/3)*(np.log(input_df[i].astype("float")) - np.log(input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float')))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [ ]:
ne_diff_summary = generate_summary_diff_df(exp_ne_df)


In [ ]:
ne_diff_summary.head()

In [ ]:
uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

In [ ]:
seir_growth_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)



seir_growth_rate.head()

In [ ]:
ne_summary = generate_summary_df(Ne_df)
ne_summary.head()

In [ ]:
test = ne_summary
test['days'] = (test.interval.astype(int) -1 )*10.76
test['date'] = dt.strptime("2024-05-01",  "%Y-%m-%d") - test.days.map(timedelta)
#test.date = test.date.astype(str)

In [ ]:
test = test.reset_index()

In [ ]:
rolling_ne =test.reset_index().groupby(["deme"])["mean_Ne_log", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50", "lower_hpd_linear_50", "upper_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"  ].rolling(4, min_periods =1).mean().reset_index()
smoothed_ne = pd.merge(rolling_ne, test, left_on = "level_1",right_index = True, how = "right")

In [ ]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line = alt.Chart(smoothed_ne).mark_area(interpolate='monotone', opacity = 0.8).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_log_50_x',axis=alt.Axis(title="Log(Effective Population Size)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_50_x' ),
    color=alt.Color('deme_x:N', title = "Region") 
).properties(
    width=850,
    height=400
)
band = alt.Chart(smoothed_ne).mark_area(interpolate='monotone', opacity = 0.5).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_log_95_x',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_log_95_x' ),
    color=alt.Color('deme_x:N') 
).properties(
    width=850,
    height=400
)

line  

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )


In [ ]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line = alt.Chart(smoothed_ne).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%b %Y"), scale=alt.Scale(domain=("2022-02-01", "2024-05-01"))),
    alt.Y('lower_hpd_linear_50_x',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50_x' ),
    color=alt.Color('deme_x:N', title = "Region") 
).properties(
    width=850,
    height=300
)
band = alt.Chart(smoothed_ne).mark_area(interpolate='monotone', opacity = 0.2).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%b %Y")),
    alt.Y('lower_hpd_linear_x',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_x' ),
    color=alt.Color('deme_x:N') 
).properties(
    width=850,
    height=300
).transform_filter(
     (datum.upper_hpd_linear_x < 150) 
 )

line  

# band = alt.Chart(smoothed_ne).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear_x'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear_x'),
#     color=alt.Color('deme_x:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
ne_plot = line 
(ne_plot).configure_axis(
    labelFontSize= 15,
    titleFontSize=20
)


In [ ]:
smoothed_ne.head()

In [ ]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "f_migration" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [ ]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)


In [ ]:
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)


In [ ]:
burnin_percent = 0.6
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "f_migrationRatesSkyline." in i:
            deme = i.split(".")[1]
            #print(deme)
            #interval = 
            #print(interval
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "mean_mig":mean_log, 
                                                   "median_mig" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [ ]:
mig_summary = generate_summary_df(mig_df_f).reset_index()
mig_summary.head()

In [ ]:
mig_summary["origin"] = mig_summary.deme.str.split("_").str[0].tolist()
mig_summary["destination"] = mig_summary.deme.str.split("_").str[2].tolist()
mig_summary.head()

In [ ]:
matrix = mig_summary.pivot(index='origin', columns='destination', values='mean_mig')
matrix = matrix.fillna(0)
matrix.head()

In [ ]:
def calc_backwards_mig_rates(mig_df_f, targetName):    
    mig_rates = {}
    target = targetName
    target_columns = []
    ne_interval= 0
    mig_list = []

    for name in mig_df_f.columns:
        try:
            if target in name.split("_")[3]:
                target_columns.append(name)
        except:
            pass

    for name in exp_ne_df.columns:
        if target in name:
            ne_interval+=1

    for interval in range(1, ne_interval,1):
        mig_rates[str(interval)+ "." + "bmr" + "." +  str(target)] = []
        for index_1, row_1 in mig_df_f.iterrows():
            mig_list = []
            for direction in target_columns:
                #print(direction)
                other_loc = direction.split(".")[1].split("_")[0]
                #print(other_loc)
                mig_rate_b = pd.to_numeric(row_1[direction]) * ((pd.to_numeric(exp_ne_df.loc[index_1,"SkylineNe."+ str(other_loc) + "." + str(interval)]))/(pd.to_numeric(exp_ne_df.loc[index_1, "SkylineNe."+ str(target) + "." + str(interval)])))
                mig_list.append(mig_rate_b)
            combined_mig = np.sum(np.array(mig_list))
            mig_rates[str(interval)+ "." + "bmr" + "." + str(target)].append(combined_mig)

    return(mig_rates) 

In [ ]:
#### if the forward migration rate is 
#### from a -> b then the backwards migration rate is from b - > a and interpreted as the rate
#### at which a lineage in b has originated from a
#### so our target is not necessarily location a but rather location b.

unique_column= []
unique_target = []
for name in mig_df_f.columns:
    try:
        if name.split("_")[3] not in unique_target:
            #print(name.split("_")[3])
            unique_column.append(name)
            unique_target.append(name.split("_")[3])
    except:
        pass

In [ ]:
mig_rates = []
for target in unique_target:
        mig_rates.append(calc_backwards_mig_rates(mig_df_f, target))


In [ ]:
mr_b_df = pd.DataFrame()
for x in mig_rates:
    x_df = pd.DataFrame(x)
    mr_b_df = pd.concat([mr_b_df, x_df], axis=1)
mr_b_df.head()

In [ ]:
## percentage of cases due to introductions

In [ ]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        
        interval = i.split(".")[0]
        deme = i.split(".")[2]

        try:
            temp_df["total."+ str(interval)] = seir_growth_rate["Ne." + str(deme) + ".diff." + str(interval)].astype("float") +  input_df[i].astype("float")
            new_df["intro.percent"+"." + str(deme) + "." + str(interval)] = input_df[i].astype("float").div(temp_df["total."+ str(interval)], axis = 0) 

        except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
            pass 
                  
    return(new_df)

In [ ]:
percent_df = generate_percent_intro_df(mr_b_df)


In [ ]:
percent_df.head()

In [ ]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            deme = i.split(".")[2]
            interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [ ]:
summary_percent_df = generate_summary_df(percent_df)

In [ ]:
summary_percent_df = summary_percent_df.reset_index()
summary_percent_df.head()

In [ ]:
#making sure that any numbers >1 are excluded as outliers from stochastics noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )
percent_df.head()

In [ ]:
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-1)*10.76 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-05-01",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.date = summary_percent_df.date.astype(str)
summary_percent_df.deme = summary_percent_df.deme.replace({"CentralEurope": "Central Europe", "NorthAmerica": "North America", "SouthAmerica":"South America", "SouthernEurope":"Southern Europe", "WesternEurope": "Western Europe" })

In [ ]:

#smoothing via 2 week rolling average 
rolling_final_percent =summary_percent_df.groupby(["deme"])["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50" ].rolling(2, min_periods =1).mean().reset_index()
#rolling_final_percent =final_north_df.groupby(["deme"])["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50" ].rolling(2, min_periods =1).mean().reset_index()
smoothed_percent = pd.merge(rolling_final_percent, summary_percent_df, left_on = "level_1",right_index = True, how = "right")

In [ ]:
line1 = alt.Chart(smoothed_percent).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y"), scale=alt.Scale(domain=("2022-02-01", "2024-05-01"))),
    alt.Y('lower_hpd_log_50_x',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', values = list(np.arange (0, 1, 0.1))), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_50_x' ), 
    alt.Color("deme_x",title = "Region")
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_50_x >0) & (datum.upper_hpd_log_50_x < 1) & (datum.upper_hpd_log_95_x < 1)
)
line1

In [ ]:
band1 = alt.Chart(smoothed_percent).mark_area(
    opacity=0.3, interpolate='monotone', clip = True
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False)),
    alt.Y('lower_hpd_log_95_x',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False)),
    alt.Y2('upper_hpd_log_95_x'),
    alt.Color("deme_x")
).properties(
    width=850,
    height=300
).transform_filter(
    (datum.lower_hpd_log_95_x >0) & (datum.upper_hpd_log_95_x < 0.9)
)
percent_plot= band1 + line1
(percent_plot).configure_axis(
    labelFontSize=20,
    titleFontSize=20
)

band1 +line1